In [1]:
import numpy as np
import sklearn
import os
import random
import math

from tqdm import tqdm
from scipy.io import loadmat

np.random.seed(1)


# Data Processing

In [2]:
full_train_data = loadmat('Data/sarcos_inv.mat')['sarcos_inv']
full_test_data = loadmat('Data/sarcos_inv_test.mat')['sarcos_inv_test']

In [3]:
def getData (full_train_data, full_test_data, num_train, num_validation, num_test):
    
    idx_train = np.random.randint(full_train_data.shape[0], size=num_train)
    idx_validation = np.random.randint(full_train_data.shape[0], size=num_validation)
    idx_test = np.random.randint(full_test_data.shape[0], size=num_test)
    
    train_data = full_train_data[idx_train, : ]
    validation_data = full_train_data[idx_validation, :]
    test_data = full_test_data[idx_test, : ]
    return (train_data, validation_data, test_data)

def Sep_X_and_Y(data, x_dim, y_dim):
    X = data[:, :x_dim]
    Y = data[:, x_dim: x_dim + y_dim ]
    return X, Y

def Unfold_Y(X, Y):
    num_tasks = Y.shape[1]
    X_new = np.vstack([X]*num_tasks)
    Y_new = np.ndarray.flatten(Y, 'F')
    T = np.arange(num_tasks)
    T_new = np.repeat(T, X.shape[0])
    return X_new, Y_new, T_new

def append_one(X):
    n = len(X)
    ones = np.zeros((n, 1)) + 1
    X_new = np.concatenate([ones, X], axis= 1)
    return X_new

def normalize_input(X):
    A = (X - np.mean(X, axis=0))
    B = A/np.std(A, axis=0)
    return B

def normalize_Y_matrix(Y):
    A = (Y - np.mean(Y, axis=0))
    return Y

In [4]:
(train_data, validation_data, test_data) = getData(full_train_data, full_test_data, 500, 100, 500)

X_train_org, Y_train_org = Sep_X_and_Y(train_data, 21, 2)
X_valid_org, Y_valid_org = Sep_X_and_Y(validation_data, 21, 2)
X_test_org , Y_test_org  = Sep_X_and_Y(test_data, 21, 2)


X_train, Y_train, T_train = Unfold_Y(X_train_org, Y_train_org)
X_valid, Y_valid, T_valid = Unfold_Y(X_valid_org, Y_valid_org)
X_test , Y_test , T_test  = Unfold_Y(X_test_org , Y_test_org )


X_train = append_one(X_train)
X_valid = append_one(X_valid)
X_test = append_one(X_test)

# Multitask Gaussian Process

In [5]:
def Kernel_input(x1, x2):  # K_input part of ICM kernel
    numerator    = 2*np.dot(np.dot(x1, Sigma_u), x2 )
    denominator1 = 1 + 2*np.dot(np.dot(x1, Sigma_u), x1 )
    denominator2 = 1 + 2*np.dot(np.dot(x2, Sigma_u), x2 )
    denominator  = math.sqrt(denominator1*denominator2) # See "Computing with infinite networks" for calculation of Expectation term 
    
    Expectation_term = (2/np.pi) * math.asin( numerator/denominator )
    similarity       = C_term +  Expectation_term  # See "Multitask Neural networks meet Multitask Gaussian Process" Paper for notation of C-term and Expectation term
    return similarity 


def Kernel_task (t1, t2):   # K_task part of the ICM Kernels
    return Omega2[t1, t2]


def Kernel(x1, t1, x2, t2): # ICM Kernel - product of input and task dependent components
    return Kernel_input(x1, x2)*Kernel_task(t1, t2)


def mtgp_fit (X_train, T_train, Y_train, Noise_variance ): # Simple MTGP implementation with specified Kernels (here ICM kernels)
    N = len(X_train)
    K = np.zeros((N, N))
            
    for i in range(0, N):
        for j in range(0, N):
            K[i, j] = Kernel(X_train[i], T_train[i], X_train[j], T_train[j] )
            
    B = np.zeros((N, N))
    for i in range(0, N):
        B[i, i] = Noise_variance[T_train[i]]
         
    
    C = K + B
    C_inv = np.linalg.inv(C)
    alpha = np.dot(C_inv, Y_train)
    model = {
        "X_train" :  X_train,
        "T_train" :  T_train,
        "Y_train" :  Y_train,
        "Noise_variance" : Noise_variance,
        "C_inv"   :  C_inv,
        "alpha"   :  alpha,
    }
    
    return model



def mtgp_predict(X, T, model):
    
    n_train  = len(model["X_train"])
    n        = len(X)
    K        = np.zeros((n_train, n))
    
    for i in range(0, n_train):
        for j in range(0, n):
            K[i, j] = Kernel( model["X_train"][i], model["T_train"][i], X[j] , T[j] )
            
    y_pred = np.dot(K.T, model["alpha"] )
    return y_pred


from sklearn.metrics import mean_squared_error


def get_stats (y_true, y_predict):
    msr = mean_squared_error(y_true, y_predict)
    stats ={
        "msr" : msr
    }
    
    return stats
    

# Hyperparameters

In [6]:
# Temporary variables for convineance - remove after tuning

temp_22_vec =  np.zeros(22) + 1
temp_2x2    =  np.identity(2)
temp_2      =  [1e-5, 1e-5]

# Kernel Hyperparameters - Prior Hyperparameters

Sigma_u =  np.diag( temp_22_vec ) # diagonal matrix of size = dimensions of X (current example: 21)
C_term  =  1             # Proportionality Constant term for bias variance 
Omega2  =  temp_2x2      # Covariance between tasks - Symmetric matrix of size T X T 


# Noise Variance - Likelihood Hyperparameters
Noise_variance = temp_2 # Noise variance - Vector of size T, number of tasks 


# Ranges for hyperparameter

Noise_variance = (1e-1, 1e-8) for both -> log scale
Sigma_u = (1e-6, 1e-2) -> log scale
C_term -> No changes, fix to be 1
Omega2 = -0.2 = (0, 2) for a11 and a22 and (-1.5, 1.5) for a21


# Bayesian Optimization

In [7]:
def get_Negative_Loss (
    Sigma_u1,
    Sigma_u2,
    Sigma_u3,
    Sigma_u4,
    Sigma_u5,
    Sigma_u6,
    Sigma_u7,
    Sigma_u8,
    Sigma_u9,
    Sigma_u10,
    Sigma_u11,
    Sigma_u12,
    Sigma_u13,
    Sigma_u14,
    Sigma_u15,
    Sigma_u16,
    Sigma_u17,
    Sigma_u18,
    Sigma_u19,
    Sigma_u20,
    Sigma_u21,
    Sigma_u22,
    
    Omega11, 
    Omega12, 
    Omega22,
    
    Noise_variance1,
    Noise_variance2 
):
    
    global Sigma_u
    Sigma_u = np.diag ( 
        np.array(
    [ 
     math.pow(10, Sigma_u1),
     math.pow(10, Sigma_u2),
     math.pow(10, Sigma_u3),
     math.pow(10, Sigma_u4),
     math.pow(10, Sigma_u5),
     math.pow(10, Sigma_u6),
     math.pow(10, Sigma_u7),
     math.pow(10, Sigma_u8),
     math.pow(10, Sigma_u9),
     math.pow(10, Sigma_u10),
     math.pow(10, Sigma_u11),
     math.pow(10, Sigma_u12),
     math.pow(10, Sigma_u13),
     math.pow(10, Sigma_u14),
     math.pow(10, Sigma_u15),
     math.pow(10, Sigma_u16),
     math.pow(10, Sigma_u17),
     math.pow(10, Sigma_u18),
     math.pow(10, Sigma_u19),
     math.pow(10, Sigma_u20),
     math.pow(10, Sigma_u21),
     math.pow(10, Sigma_u22)
    ] )
    )
        
    global Omega2
    Omega2 = np.array( [[Omega11, Omega12], [Omega12, Omega22]] )
    
    
    global Noise_variance
    Noise_variance = [math.pow(10, Noise_variance1) , math.pow(10, Noise_variance2)]
    
    
    
    model = mtgp_fit (X_train, T_train, Y_train, Noise_variance )
    predict = mtgp_predict(X_valid, T_valid, model)
    stats = get_stats(Y_valid, predict)
    return -stats['msr']


In [8]:
from bayes_opt import BayesianOptimization


pbounds = {
                "Sigma_u1" : (-6, -2),
                "Sigma_u2" : (-6, -2),
                "Sigma_u3" : (-6, -2),
                "Sigma_u4" : (-6, -2),
                "Sigma_u5" : (-6, -2),
                "Sigma_u6" : (-6, -2),
                "Sigma_u7" : (-6, -2),
                "Sigma_u8" : (-6, -2),
                "Sigma_u9" : (-6, -2),
                "Sigma_u10" : (-6, -2),
                "Sigma_u11" : (-6, -2),
                "Sigma_u12" : (-6, -2),
                "Sigma_u13" : (-6, -2),
                "Sigma_u14" : (-6, -2),
                "Sigma_u15" : (-6, -2),
                "Sigma_u16" : (-6, -2),
                "Sigma_u17" : (-6, -2),
                "Sigma_u18" : (-6, -2),
                "Sigma_u19" : (-6, -2),
                "Sigma_u20" : (-6, -2),
                "Sigma_u21" : (-6, -2),
                "Sigma_u22" : (-6, -2),

                "Omega11" : (0, 2) ,
                "Omega12" : (-1.5, 1.5), 
                "Omega22" : (0, 2) ,

                "Noise_variance1" : (-1, -8),
                "Noise_variance2" : (-1, -8) 
          }


optimizer = BayesianOptimization(
    f= get_Negative_Loss,
    pbounds=pbounds,
    random_state=1,
)



In [12]:
optimizer.maximize(
    init_points=10,
    n_iter=100,
)

|   iter    |  target   | Noise_... | Noise_... |  Omega11  |  Omega12  |  Omega22  | Sigma_u1  | Sigma_u10 | Sigma_u11 | Sigma_u12 | Sigma_u13 | Sigma_u14 | Sigma_u15 | Sigma_u16 | Sigma_u17 | Sigma_u18 | Sigma_u19 | Sigma_u2  | Sigma_u20 | Sigma_u21 | Sigma_u22 | Sigma_u3  | Sigma_u4  | Sigma_u5  | Sigma_u6  | Sigma_u7  | Sigma_u8  | Sigma_u9  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  111      | -424.4    | -2.298    | -7.532    |  1.872    | -0.6494   |  0.5493   | -3.794    | -5.259    | -4.232    | -5.259    | -2.21     | -4.025    | -2.144    | -3.42     | -4.063    | -2.129    | -5.427    | -3.471    | -3.352    | -4.265    | -5.763    | -4.029    | -4.139    | -2.773    

|  124      | -157.4    | -1.0      | -1.0      |  0.123    | -0.08016  |  1.178    | -3.398    | -4.859    | -4.264    | -4.52     | -4.029    | -5.895    | -4.772    | -4.756    | -4.022    | -3.354    | -3.603    | -3.128    | -5.021    | -4.717    | -3.319    | -3.678    | -5.276    | -3.445    | -4.287    | -3.37     | -2.619    | -3.367    |
|  125      | -115.4    | -1.0      | -1.0      |  0.7967   | -0.8113   |  1.074    | -2.448    | -3.917    | -4.433    | -4.8      | -3.491    | -4.404    | -4.916    | -3.106    | -3.879    | -2.041    | -3.641    | -4.245    | -5.948    | -4.078    | -2.185    | -4.133    | -4.609    | -3.18     | -3.907    | -3.132    | -4.223    | -4.787    |
|  126      | -65.84    | -1.0      | -1.0      |  1.823    | -0.3604   |  0.5656   | -3.8      | -5.39     | -4.328    | -4.492    | -5.156    | -4.659    | -3.6      | -2.738    | -2.903    | -4.208    | -5.732    | -5.271    | -3.203    | -4.872    | -3.384    | -4.323    | -4.156    | -2.375    

|  139      | -73.66    | -1.0      | -1.0      |  0.7449   |  0.3279   |  0.6237   | -4.403    | -4.246    | -3.863    | -2.709    | -4.3      | -4.893    | -3.562    | -3.238    | -3.803    | -5.018    | -4.085    | -4.389    | -2.496    | -4.39     | -3.522    | -3.632    | -4.045    | -2.856    | -3.739    | -3.914    | -4.639    | -3.493    |
|  140      | -74.14    | -1.0      | -1.0      |  1.538    | -0.4      |  0.3249   | -2.805    | -4.403    | -3.604    | -5.611    | -4.113    | -5.177    | -5.355    | -2.786    | -3.723    | -4.309    | -3.032    | -3.782    | -5.115    | -3.75     | -3.131    | -2.934    | -5.483    | -2.775    | -4.519    | -3.014    | -3.585    | -2.623    |
|  141      | -160.5    | -1.0      | -1.0      |  0.5421   | -0.7976   |  0.5993   | -3.299    | -5.255    | -3.807    | -5.193    | -4.766    | -3.644    | -3.867    | -2.206    | -4.058    | -4.861    | -3.417    | -3.873    | -4.295    | -2.23     | -3.642    | -3.095    | -5.227    | -4.245    

|  154      | -130.6    | -1.0      | -1.0      |  0.299    | -0.4722   |  1.152    | -3.418    | -4.941    | -3.826    | -3.118    | -4.152    | -4.008    | -5.151    | -4.057    | -3.142    | -2.939    | -4.761    | -4.403    | -4.546    | -2.727    | -2.56     | -3.326    | -3.832    | -3.016    | -4.203    | -3.362    | -3.769    | -3.859    |
|  155      | -63.2     | -1.0      | -1.0      |  1.111    |  0.3617   |  1.159    | -3.285    | -5.065    | -3.101    | -3.487    | -4.182    | -3.355    | -2.682    | -3.477    | -3.832    | -4.182    | -2.37     | -3.533    | -4.298    | -4.849    | -4.17     | -4.373    | -4.072    | -4.344    | -4.626    | -2.09     | -3.893    | -3.193    |
|  156      | -59.81    | -1.0      | -1.0      |  1.383    |  0.8088   |  1.174    | -4.661    | -4.13     | -4.744    | -4.248    | -3.449    | -4.342    | -4.448    | -2.788    | -3.737    | -3.318    | -3.277    | -5.04     | -3.635    | -4.424    | -3.933    | -3.308    | -3.3      | -2.518    

|  169      | -94.85    | -1.0      | -1.0      |  0.8368   | -0.4066   |  1.153    | -4.396    | -4.591    | -4.491    | -3.218    | -3.758    | -4.629    | -4.59     | -3.918    | -5.253    | -4.105    | -5.538    | -3.435    | -5.105    | -3.459    | -3.836    | -3.207    | -5.365    | -4.264    | -4.099    | -3.865    | -2.703    | -3.792    |
|  170      | -59.79    | -1.0      | -1.0      |  1.203    |  0.4222   |  1.708    | -3.807    | -3.489    | -3.201    | -5.291    | -5.255    | -5.316    | -4.058    | -2.946    | -3.58     | -4.768    | -4.866    | -3.465    | -5.553    | -3.731    | -3.053    | -3.452    | -5.153    | -2.325    | -5.036    | -3.888    | -4.926    | -3.313    |
|  171      | -62.71    | -1.0      | -1.0      |  1.052    |  0.3454   |  1.772    | -4.224    | -4.564    | -2.787    | -4.292    | -4.963    | -2.909    | -5.736    | -4.463    | -4.52     | -4.767    | -4.31     | -4.413    | -4.083    | -4.198    | -4.227    | -5.547    | -4.18     | -3.158    

|  184      | -102.2    | -1.0      | -1.0      |  0.6711   | -0.8543   |  1.243    | -4.74     | -5.289    | -4.83     | -4.839    | -5.053    | -3.609    | -3.891    | -2.846    | -2.755    | -4.978    | -4.449    | -5.092    | -4.982    | -4.625    | -2.843    | -4.373    | -4.507    | -4.193    | -5.027    | -2.432    | -3.854    | -2.43     |
|  185      | -84.53    | -1.0      | -1.0      |  1.077    |  0.6142   |  0.521    | -2.624    | -5.823    | -2.932    | -4.165    | -4.027    | -5.975    | -4.63     | -3.259    | -4.003    | -3.563    | -4.514    | -4.17     | -3.637    | -3.458    | -2.818    | -4.175    | -2.597    | -3.349    | -5.056    | -3.389    | -2.731    | -2.599    |
|  186      | -202.4    | -1.0      | -1.0      |  0.7149   |  0.6083   |  0.3303   | -4.112    | -4.862    | -3.698    | -4.788    | -3.449    | -4.427    | -3.489    | -3.095    | -3.162    | -4.635    | -5.04     | -4.713    | -5.135    | -4.441    | -2.768    | -3.035    | -4.298    | -2.221    

|  199      | -95.92    | -1.0      | -1.0      |  0.2515   | -0.01527  |  1.185    | -2.597    | -4.083    | -3.766    | -5.395    | -5.648    | -3.367    | -5.805    | -4.198    | -4.332    | -5.063    | -4.722    | -4.92     | -4.712    | -4.487    | -3.155    | -3.574    | -4.459    | -3.929    | -4.551    | -4.488    | -3.018    | -3.68     |
|  200      | -88.39    | -1.0      | -1.0      |  0.7194   | -0.436    |  1.113    | -2.865    | -4.794    | -4.475    | -4.688    | -3.707    | -4.823    | -4.4      | -3.941    | -4.049    | -3.867    | -4.8      | -3.804    | -5.979    | -2.593    | -2.839    | -3.334    | -4.122    | -3.537    | -5.725    | -4.891    | -3.997    | -3.175    |
|  201      | -80.84    | -1.0      | -1.0      |  1.416    | -1.092    |  1.365    | -3.705    | -4.92     | -2.7      | -4.189    | -4.757    | -5.264    | -3.741    | -3.362    | -4.527    | -3.877    | -4.158    | -4.112    | -3.441    | -4.59     | -3.756    | -4.487    | -5.605    | -5.146    

|  214      | -442.9    | -1.0      | -1.0      |  0.1567   | -0.5082   |  1.295    | -2.359    | -5.673    | -3.882    | -4.447    | -4.702    | -5.429    | -3.705    | -4.869    | -2.85     | -3.067    | -4.731    | -5.186    | -4.548    | -3.89     | -3.961    | -4.569    | -4.807    | -3.556    | -4.268    | -2.8      | -3.565    | -3.658    |
|  215      | -69.65    | -1.0      | -1.0      |  1.727    | -0.5856   |  0.6578   | -4.137    | -3.718    | -2.215    | -5.288    | -4.325    | -4.562    | -5.298    | -3.403    | -3.217    | -4.484    | -4.288    | -4.464    | -3.544    | -2.3      | -4.011    | -3.29     | -3.02     | -4.113    | -4.297    | -3.154    | -5.2      | -3.761    |
|  216      | -113.1    | -1.0      | -1.0      |  0.8814   | -1.012    |  0.8694   | -3.263    | -4.9      | -5.075    | -3.606    | -4.837    | -5.59     | -4.312    | -4.11     | -3.483    | -3.691    | -5.201    | -3.973    | -4.839    | -3.41     | -3.618    | -2.52     | -5.069    | -4.334    

In [13]:
print(optimizer.max)      

{'target': -30.146467090950367, 'params': {'Noise_variance1': -3.9300461803486906, 'Noise_variance2': -5.784254301164108, 'Omega11': 0.7959812788158449, 'Omega12': 0.017226052352431243, 'Omega22': 0.3791034056678546, 'Sigma_u1': -2.1400443035563446, 'Sigma_u10': -4.823137247022981, 'Sigma_u11': -5.586161775534395, 'Sigma_u12': -5.422738399083808, 'Sigma_u13': -5.943630823230228, 'Sigma_u14': -3.136217202150422, 'Sigma_u15': -3.7420067190365036, 'Sigma_u16': -2.8216865952609913, 'Sigma_u17': -3.9716803068634543, 'Sigma_u18': -2.8327158419833847, 'Sigma_u19': -3.2169431429126423, 'Sigma_u2': -2.8886060925632986, 'Sigma_u20': -4.374068524091655, 'Sigma_u21': -3.4089174672500753, 'Sigma_u22': -5.280822809228102, 'Sigma_u3': -4.712720146448384, 'Sigma_u4': -5.30958151952083, 'Sigma_u5': -4.3654510999469505, 'Sigma_u6': -5.034325008137605, 'Sigma_u7': -4.372312023482568, 'Sigma_u8': -2.0991106302507516, 'Sigma_u9': -4.718722696711875}}


# Single Layer Neural Network

In [ ]:
from sklearn.neural_network import MLPRegressor

num_hidd_units = 700

for rs in range(0, 1):
    clf = MLPRegressor(hidden_layer_sizes= (num_hidd_units, ), activation='logistic' ,random_state=rs,  solver='adam', max_iter=100000)
    clf.fit(X_train_org, Y_train_org)
    Y_pred_NN = clf.predict(X_valid_org)
    Y_pred_NN1 = Y_pred_NN.flatten()
    Y_valid_org1 = Y_valid_org.flatten()
    
    stats = get_stats(Y_pred_NN1, Y_valid_org1)
    print( -stats['msr'] )
    
    



In [ ]:
math.pow(10, -0.7248)